In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# # Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
data = pd.read_csv('mpst_full_data.csv')
data.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [4]:
# Convert to list
df = data.plot_synopsis.values.tolist()

# Remove Emails
df = [re.sub('\S*@\S*\s?', '', sent) for sent in df]

# Remove new line characters
df = [re.sub('\s+', ' ', sent) for sent in df]

# Remove distracting single quotes
df = [re.sub("\'", "", sent) for sent in df]

pprint(df[:1])

['Note: this synopsis is for the orginal Italian release with the segments in '
 'this certain order.Boris Karloff introduces three horror tales of the '
 'macabre and the supernatural known as the Three Faces of Fear.THE '
 'TELEPHONERosy (Michele Mercier) is an attractive, high-priced Parisian '
 'call-girl who returns to her spacious, basement apartment after an evening '
 'out when she immediately gets beset by a series of strange phone calls. The '
 'caller soon identified himself as Frank, her ex-pimp who has recently '
 'escaped from prison. Rosy is terrified for it was her testimony that landed '
 'the man in jail. Looking for solace, Rosy phones her lesbian lover Mary '
 '(Lynda Alfonsi). The two women have been estranged for some time, but Rosy '
 'is certain that she is the only one who can help her. Mary agrees to come '
 'over that night. Seconds later, Frank calls again, promising that no matter '
 'who she calls for protection, he will have his revenge. Unknown to Rosy, 

In [5]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ankan_rokr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

df_words = list(sent_to_words(df))

print(df_words[:1])

[['note', 'this', 'synopsis', 'is', 'for', 'the', 'orginal', 'italian', 'release', 'with', 'the', 'segments', 'in', 'this', 'certain', 'order', 'boris', 'karloff', 'introduces', 'three', 'horror', 'tales', 'of', 'the', 'macabre', 'and', 'the', 'supernatural', 'known', 'as', 'the', 'three', 'faces', 'of', 'fear', 'the', 'telephonerosy', 'michele', 'mercier', 'is', 'an', 'attractive', 'high', 'priced', 'parisian', 'call', 'girl', 'who', 'returns', 'to', 'her', 'spacious', 'basement', 'apartment', 'after', 'an', 'evening', 'out', 'when', 'she', 'immediately', 'gets', 'beset', 'by', 'series', 'of', 'strange', 'phone', 'calls', 'the', 'caller', 'soon', 'identified', 'himself', 'as', 'frank', 'her', 'ex', 'pimp', 'who', 'has', 'recently', 'escaped', 'from', 'prison', 'rosy', 'is', 'terrified', 'for', 'it', 'was', 'her', 'testimony', 'that', 'landed', 'the', 'man', 'in', 'jail', 'looking', 'for', 'solace', 'rosy', 'phones', 'her', 'lesbian', 'lover', 'mary', 'lynda', 'alfonsi', 'the', 'two', 

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[df_words[0]]])

['note', 'this', 'synopsis', 'is', 'for', 'the', 'orginal', 'italian', 'release', 'with', 'the', 'segments', 'in', 'this', 'certain', 'order', 'boris_karloff', 'introduces', 'three', 'horror', 'tales', 'of', 'the', 'macabre', 'and', 'the', 'supernatural', 'known', 'as', 'the', 'three', 'faces', 'of', 'fear', 'the', 'telephonerosy', 'michele', 'mercier', 'is', 'an', 'attractive', 'high_priced', 'parisian', 'call', 'girl', 'who', 'returns', 'to', 'her', 'spacious', 'basement', 'apartment', 'after', 'an', 'evening', 'out', 'when', 'she', 'immediately', 'gets', 'beset', 'by', 'series', 'of', 'strange', 'phone', 'calls', 'the', 'caller', 'soon', 'identified', 'himself', 'as', 'frank', 'her', 'ex', 'pimp', 'who', 'has', 'recently', 'escaped', 'from', 'prison', 'rosy', 'is', 'terrified', 'for', 'it', 'was', 'her', 'testimony', 'that', 'landed', 'the', 'man', 'in', 'jail', 'looking', 'for', 'solace', 'rosy', 'phones', 'her', 'lesbian', 'lover', 'mary', 'lynda', 'alfonsi', 'the', 'two', 'women'

In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
!python3 -m spacy download en

    100% |████████████████████████████████| 11.1MB 98.7MB/s eta 0:00:01
  Running setup.py install for en-core-web-sm ... done
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/ankan_rokr/.local/lib/python3.5/site-packages/en_core_web_sm -->
/home/ankan_rokr/.local/lib/python3.5/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [11]:
# Remove Stop Words
df_words_nostops = remove_stopwords(df_words)

# Form Bigrams
df_words_bigrams = make_bigrams(df_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
df_lemmatized = lemmatization(df_words_bigrams, allowed_postags=['ADJ', 'VERB', 'ADV'])

print(df_lemmatized[:1])

[['synopsi', 'orginal', 'italian', 'segment', 'certain', 'introduce', 'supernatural', 'know', 'fear', 'attractive', 'high_priced', 'parisian', 'return', 'spacious', 'immediately', 'get', 'strange', 'soon', 'identify', 'frank', 'recently', 'escape', 'rosy', 'terrify', 'land', 'look', 'rosy', 'lesbian', 'estrange', 'rosy', 'certain', 'agree', 'come', 'later', 'frank', 'promise', 'call', 'unknown', 'rosy', 'impersonate', 'arrive', 'rosys', 'soon', 'good', 'calm', 'give', 'strike', 'put', 'later', 'rosy', 'sleep', 'get', 'note', 'make', 'strange', 'learn', 'escape', 'know', 'rosy', 'would', 'call', 'explain', 'feel', 'come', 'back', 'busy', 'fail', 'frank', 'real', 'mary', 'rosys', 'sound', 'rosy', 'murderous', 'kill', 'wrong', 'slowly', 'make', 'rosys', 'however', 'early', 'place', 'butcher_knife', 'pillow', 'rosy', 'frank', 's', 'begin', 'rosy', 'drop', 'surround', 'former', 'young', 'long', 'find', 'behead', 'plunge', 'take', 'later', 'stop', 'small', 'rural', 'ask', 'notice', 'several'

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(df_lemmatized)

# Create Corpus
texts = df_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 4), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 3), (37, 2), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 3), (64, 1), (65, 1), (66, 2), (67, 2), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 4), (78, 1), (79, 1), (80, 2), (81, 2), (82, 1), (83, 3), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 3), (101, 2), (102, 1), (103, 1), (104, 3), (105, 1), (106, 2), (107, 1), (108, 1), (109, 4), (110, 1)

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 2) above implies, word id 0 occurs twice in the first document. Likewise, word id 1 occurs once and so on.....
This is used as the input by the LDA model.

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
lda_model

In [22]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"david" + 0.024*"mike" + 0.022*"billy" + 0.016*"michael" + '
  '0.013*"steve" + 0.013*"todd" + 0.013*"nick" + 0.012*"emily" + 0.011*"ellen" '
  '+ 0.011*"molly"'),
 (1,
  '0.014*"become" + 0.009*"live" + 0.009*"make" + 0.009*"also" + 0.009*"young" '
  '+ 0.008*"new" + 0.008*"begin" + 0.007*"first" + 0.007*"however" + '
  '0.007*"leave"'),
 (2,
  '0.039*"kill" + 0.017*"take" + 0.017*"find" + 0.010*"arrive" + '
  '0.009*"reveal" + 0.008*"leave" + 0.008*"escape" + 0.008*"use" + '
  '0.007*"dead" + 0.007*"shoot"'),
 (3,
  '0.040*"go" + 0.035*"tell" + 0.034*"get" + 0.024*"see" + 0.021*"find" + '
  '0.019*"back" + 0.017*"come" + 0.016*"leave" + 0.016*"say" + 0.016*"take"'),
 (4,
  '0.015*"use" + 0.010*"destroy" + 0.009*"find" + 0.008*"human" + '
  '0.007*"cause" + 0.006*"appear" + 0.006*"reveal" + 0.006*"create" + '
  '0.006*"red" + 0.006*"save"')]


In [23]:
doc_lda

In [24]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.904151191081055

Coherence Score:  0.3331767992985529


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      36.998875        1       1 -0.146036 -0.199328
1      30.243530        1       2 -0.135162 -0.054326
2      22.221745        1       3 -0.127393  0.067436
4       8.188994        1       4 -0.022353  0.238498
0       2.346859        1       5  0.430945 -0.052280, topic_info=      Category          Freq     Term         Total  loglift  logprob
48118  Default  27906.000000     kill  27906.000000  30.0000  30.0000
42784  Default  48982.000000       go  48982.000000  29.0000  29.0000
47335  Default  40462.000000      get  40462.000000  28.0000  28.0000
52652  Default  42721.000000     tell  42721.000000  27.0000  27.0000
55187  Default  18074.000000      say  18074.000000  26.0000  26.0000
51910  Default  16965.000000      not  16965.000000  25.0000  25.0000
50399  Default  30334.000000      see  30334.000000  24.0000  24.0000
2079   Default  13271.000000      use  13271.000000  23.0000  23.0000
8204   Default  15290.000000   become  15290.000000  22.0000  22.0000
17327  Default  26342.000000     back  26342.000000  21.0000  21.0000
24077  Default  15167.000000      ask  15167.000000  20.0000  20.0000
10336  Default  10383.000000     look  10383.000000  19.0000  19.0000
23082  Default   8659.000000        s   8659.000000  18.0000  18.0000
38117  Default  23744.000000     come  23744.000000  17.0000  17.0000
30894  Default  41984.000000     find  41984.000000  16.0000  16.0000
40120  Default   8160.000000       do   8160.000000  15.0000  15.0000
58220  Default   2101.000000    david   2101.000000  14.0000  14.0000
41969  Default   4212.000000  destroy   4212.000000  13.0000  13.0000
43401  Default   6185.000000   escape   6185.000000  12.0000  12.0000
55876  Default  12624.000000   reveal  12624.000000  11.0000  11.0000
23314  Default   2240.000000  michael   2240.000000  10.0000  10.0000
26110  Default  10025.000000     live  10025.000000   9.0000   9.0000
54626  Default   5738.000000    shoot   5738.000000   8.0000   8.0000
21627  Default   1716.000000     mike   1716.000000   7.0000   7.0000
58991  Default   1612.000000    billy   1612.000000   6.0000   6.0000
64728  Default   6027.000000     walk   6027.000000   5.0000   5.0000
9649   Default  13520.000000     want  13520.000000   4.0000   4.0000
66574  Default  10027.000000    young  10027.000000   3.0000   3.0000
43411  Default   6734.000000    cause   6734.000000   2.0000   2.0000
17185  Default   7649.000000     lead   7649.000000   1.0000   1.0000
...        ...           ...      ...           ...      ...      ...
44486   Topic5    914.246094     nick    915.503845   3.7507  -4.3719
47550   Topic5    916.031311     todd    917.298462   3.7507  -4.3700
34130   Topic5    916.179565    steve    917.461609   3.7507  -4.3698
52646   Topic5    852.198425    emily    853.459717   3.7506  -4.4422
25159   Topic5    802.328796    ellen    803.602539   3.7505  -4.5025
12775   Topic5    787.393188    molly    788.658508   3.7505  -4.5213
42396   Topic5    768.233215    kelly    769.506836   3.7504  -4.5459
3995    Topic5    738.917358     adam    740.181458   3.7504  -4.5848
34158   Topic5    661.823120    sally    663.084656   3.7502  -4.6950
56711   Topic5    622.523987     alex    623.780762   3.7501  -4.7563
43692   Topic5    654.661316    aiden    656.071960   3.7499  -4.7059
42838   Topic5    568.539917    laura    569.814575   3.7499  -4.8470
54505   Topic5    542.748352      pat    544.022644   3.7497  -4.8934
15904   Topic5    455.780029     stan    457.068817   3.7493  -5.0680
35983   Topic5    447.557220       gu    448.841187   3.7492  -5.0862
62042   Topic5    425.965424    nicky    427.223389   3.7491  -5.1357
28237   Topic5    390.413055     andy    391.704498   3.7488  -5.2228
25352   Topic5    384.104767     skye    385.376434   3.7488  -5.2391
41862   Topic5    377.186981     beth    378.450378   3.7487  -5.2573
19747   T

In [26]:
mallet_path = "Mallet/bin/mallet" # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=100, id2word=id2word)


# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(89,
  [('back', 0.05714711074246581),
   ('drive', 0.04215317212190817),
   ('hit', 0.03602798868542504),
   ('pull', 0.034879516791084454),
   ('run', 0.03271018099066335),
   ('stop', 0.028733065356557987),
   ('open', 0.023501137837895317),
   ('shoot', 0.02188476998660116),
   ('walk', 0.021034050064867395),
   ('knock', 0.017950190348582487)]),
 (6,
  [('murder', 0.04588172528419584),
   ('kill', 0.03467804817991546),
   ('suspect', 0.028481142528829974),
   ('accuse', 0.026182952353592973),
   ('arrest', 0.02523905281733492),
   ('innocent', 0.019739812040874955),
   ('laura', 0.019698772930602865),
   ('commit', 0.018467599622440184),
   ('admit', 0.01822136496080765),
   ('guilty', 0.016620839660196167)]),
 (91,
  [('dark', 0.0444233550253073),
   ('find', 0.04162147505422994),
   ('die', 0.026437093275488068),
   ('doug', 0.023318872017353578),
   ('burn', 0.021466015907447577),
   ('leave', 0.01839298626174982),
   ('call', 0.015681489515545915),
   ('bury', 0.0155007230657

NameError: name 'data_lemmatized' is not defined

In [27]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    mallet_path = "Mallet/bin/mallet"
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [28]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=df_lemmatized, start=2, limit=60, step=6)

In [29]:
model_list

In [31]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=100))

[(0,
  '0.024*"destroy" + 0.013*"human" + 0.011*"create" + 0.010*"steve" + '
  '0.009*"large" + 0.008*"alien" + 0.008*"fly" + 0.008*"find" + 0.007*"save" + '
  '0.007*"build" + 0.007*"escape" + 0.007*"send" + 0.007*"remain" + '
  '0.006*"explain" + 0.006*"holly" + 0.006*"reveal" + 0.006*"open" + '
  '0.005*"infect" + 0.005*"begin" + 0.005*"discover" + 0.005*"stop" + '
  '0.005*"locate" + 0.005*"lead" + 0.005*"survive" + 0.005*"work" + '
  '0.005*"activate" + 0.005*"explode" + 0.005*"call" + 0.005*"attack" + '
  '0.004*"control" + 0.004*"future" + 0.004*"enter" + 0.004*"reach" + '
  '0.004*"giant" + 0.004*"massive" + 0.004*"set" + 0.004*"underground" + '
  '0.004*"attempt" + 0.004*"final" + 0.003*"military" + 0.003*"original" + '
  '0.003*"complete" + 0.003*"launch" + 0.003*"manage" + 0.003*"develop" + '
  '0.003*"include" + 0.003*"entire" + 0.003*"crash" + 0.003*"main" + '
  '0.003*"medical" + 0.003*"trap" + 0.003*"secret" + 0.003*"damage" + '
  '0.003*"prevent" + 0.003*"nuclear" + 0.0

In [32]:
from tqdm import tqdm

In [33]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Initial output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in tqdm(enumerate(ldamodel[corpus])):
        row = sorted(row, key= lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel= optimal_model, corpus=corpus, texts=df)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

14828it [01:00, 243.17it/s]


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,11.0,0.1217,"find, reveal, leave, discover, real, follow, m...",Note: this synopsis is for the orginal Italian...
1,1,7.0,0.2980,"fight, evil, save, great, free, lead, find, da...","Two thousand years ago, Nhagruul the Foul, a s..."
2,2,9.0,0.2020,"shoot, back, drive, make, arrive, give, hit, l...","Matuscheks, a gift store in Budapest, is the w..."
3,3,15.0,0.2658,"play, show, billy, win, include, end, perform,...","Glenn Holland, not a morning person by anyones..."
4,4,10.0,0.1919,"make, back, give, good, talk, leave, walk, sta...","In May 1980, a Cuban man named Tony Montana (A..."
5,5,10.0,0.1350,"make, back, give, good, talk, leave, walk, sta...",George Falconer (Colin Firth) approaches a car...
6,6,18.0,0.1140,"joe, local, meet, call, leave, work, pay, alex...",Baise-moi tells the story of Nadine and Manu w...
7,7,11.0,0.1444,"find, reveal, leave, discover, real, follow, m...",Kyle Pratt (Jodie Foster) is a propulsion engi...
8,8,16.0,0.1952,"kill, david, steal, nick, reveal, give, make, ...",Small-time Italian-American criminals Caesar E...
9,9,18.0,0.3023,"joe, local, meet, call, leave, work, pay, alex...",The movie begins with a video being shot of me...


In [34]:
df_dominant_topic['Keywords'][1]

'fight, evil, save, great, free, lead, find, dark, return, fall'

In [51]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.4851,"destroy, human, create, steve, large, alien, f...",=== Setting and characters === Mass Effect 2 i...
1,1.0,0.5769,"frank, back, leave, find, begin, make, give, a...",The central character and narrator is 17-year-...
2,2.0,0.5134,"kill, dead, find, attack, die, escape, arrive,...",Dragon Ball Z picks up five years after the en...
3,3.0,0.4433,"young, begin, rise, die, live, elizabeth, suff...",The play consists of four interconnecting plot...
4,4.0,0.6961,"back, open, walk, suddenly, run, turn, pull, b...",1. A miracle in the woodsIn the depth of the f...


In [52]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,11.0,"find, reveal, leave, discover, real, follow, m...",1026.0,0.0692
1,7.0,"fight, evil, save, great, free, lead, find, da...",543.0,0.0366
2,9.0,"shoot, back, drive, make, arrive, give, hit, l...",767.0,0.0517
3,15.0,"play, show, billy, win, include, end, perform,...",765.0,0.0516
4,10.0,"make, back, give, good, talk, leave, walk, sta...",635.0,0.0428
5,10.0,"make, back, give, good, talk, leave, walk, sta...",1110.0,0.0749
6,18.0,"joe, local, meet, call, leave, work, pay, alex...",865.0,0.0583
7,11.0,"find, reveal, leave, discover, real, follow, m...",862.0,0.0581
8,16.0,"kill, david, steal, nick, reveal, give, make, ...",491.0,0.0331
9,18.0,"joe, local, meet, call, leave, work, pay, alex...",583.0,0.0393


In [55]:
data['New'] = df_dominant_topics['Topic_Keywords']

In [56]:
data['New'][0]

'find, reveal, leave, discover, real, follow, miss, murder, investigate, realize'

In [57]:
data.to_csv('data.csv', index=True)

In [58]:
# function for cleaning the plots of the movies
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    #text = re.sub('\W', ' ', text)
    #text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

# function for text cleaning 
def cleaned(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [59]:
data['cleaned'] = list(data['New'].apply(clean_text))

In [60]:
data['cleaned'] = list(data['New'].apply(cleaned))

In [61]:
data['cleaned'][0]

'find reveal leave discover real follow miss murder investigate realize'

In [62]:
new = data[['cleaned', 'tags', 'split']]

In [63]:
new.head()

,cleaned,tags,split
0,find reveal leave discover real follow miss mu...,"cult, horror, gothic, murder, atmospheric",train
1,fight evil save great free lead find dark retu...,violence,train
2,shoot back drive make arrive give hit leave st...,romantic,test
3,play show billy win include end perform make f...,"inspiring, romantic, stupid, feel-good",train
4,make back give good talk leave walk start feel...,"cruelty, murder, dramatic, cult, violence, atm...",val


In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

data['cleaned'] = data['cleaned'].apply(lambda x: remove_stopwords(x))

In [64]:
train_new =  new['split']=='train'
train = new[train_new]

   
    
test_new =  new['split']=='test'
test = new[test_new]

  
    
validation_new =  new['split']=='val'
val = new[validation_new]


In [65]:
vectorizer_tags = CountVectorizer(tokenizer = lambda x: x.split(','), binary='true')
y_train = vectorizer_tags.fit_transform(train['tags'])
y_test = vectorizer_tags.transform(test['tags'])

In [66]:
start = datetime.now()
# vectorizer_unigram = TfidfVectorizer(smooth_idf=True, norm="l2", \
#                              tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,1))
x_train_multilabel_unigram = vectorizer_tags.fit_transform(train['cleaned'])
x_test_multilabel_unigram = vectorizer_tags.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:00.048030


In [67]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_unigram,y_train)
predictions = classifier.predict (x_test_multilabel_unigram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.4268802294549495
Micro-average quality numbers
Precision: 0.0300, Recall: 0.6040, F1-measure: 0.0572
Macro-average quality numbers
Precision: 0.0299, Recall: 0.5991, F1-measure: 0.0507
              precision    recall  f1-score   support

           0       0.01      0.54      0.03        35
           1       0.06      0.72      0.11       117
           2       0.02      0.85      0.03        26
           3       0.01      0.64      0.01        11
           4       0.01      0.69      0.01        13
           5       0.01      0.65      0.03        26
           6       0.01      0.33      0.02        15
           7       0.03      0.56      0.06        75
           8       0.00      1.00      0.00         4
           9       0.01      0.61      0.03        31
          10       0.00      0.25      0.00         8
          11       0.01      0.45      0.01        20
          12       0.03      0.67      0.06        79
          13       0.00    

In [68]:
start = datetime.now()
vectorizer_unigram = TfidfVectorizer(smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(6,6))
x_train_multilabel_unigram = vectorizer_unigram.fit_transform(train['cleaned'])
x_test_multilabel_unigram = vectorizer_unigram.transform(test['cleaned'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:00.111581


In [69]:
start = datetime.now()
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l2', class_weight="balanced"), n_jobs=-1)
classifier.fit(x_train_multilabel_unigram,y_train)
predictions = classifier.predict (x_test_multilabel_unigram)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.42767088030543343
Micro-average quality numbers
Precision: 0.0302, Recall: 0.6088, F1-measure: 0.0575
Macro-average quality numbers
Precision: 0.0299, Recall: 0.5999, F1-measure: 0.0508
              precision    recall  f1-score   support

           0       0.01      0.46      0.02        35
           1       0.06      0.72      0.11       117
           2       0.02      0.65      0.04        26
           3       0.01      0.64      0.01        11
           4       0.01      0.69      0.01        13
           5       0.01      0.65      0.02        26
           6       0.01      0.33      0.02        15
           7       0.03      0.56      0.06        75
           8       0.00      1.00      0.00         4
           9       0.01      0.58      0.03        31
          10       0.00      0.38      0.00         8
          11       0.01      0.45      0.01        20
          12       0.03      0.70      0.06        79
          13       0.00   